<a href="https://colab.research.google.com/github/masoudinejad/dt_pruning/blob/main/orap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimized Resource Aware Pruning (ORAP)

In this session you will see:
- [1 - Resource demand calculation of decision trees](#1)
- [2 - The general concept of global optimization](#2) 
- [3 - ORAP algorithm](#3)
- [4 - ORAP Performance](#4)
- [5 - ORAP vs CCP](#5)

<a name="1"></a>
## 1 - Decision tree resource demand


Different type of resources

Formulation of the APD

<a name="2"></a>
## 2 - Pruning as an optimization problem 

<a name="2.1"></a>
## 2.1 - Average Probabilistic Depth (APD)

APD can be calculated from two different formulation. Initially it is explained using the naming by finding the depth of each leaf and probability of reaching that leaf from the relative number of samples in that leaf.

Number of samples in a node $n$ is shown using $\sigma(n)$, accordingly, $\left|\sigma\right|$ shows the total number of samples or number of samples in the root node.
Each leaf is shown using $l$ while set of all leaves is $\Lambda$. Therefore, probability of reaching each leaf is defined as:
$$
\mathbb{P}(l)=\frac{\sigma(l)}{\left|\sigma\right|}
$$

Depth of a leaf is defined by the number of inner-node necessary to reach that leaf. To be noted that the leaf itself is not counted. This is due to the fact that APD is to formulated to quantize the resource needs for the inference and a leaf does not require a comparison.

When the depth of a leaf $l$ is shown using $d_l$, APD of a decision tree $T$ can be formulated as:
$$
A(T) = \sum\limits_{\forall l \in \Lambda}\left(\mathbb{P}(l) \cdot d_l\right)
$$

This formulation explains the calculation of the APD in a very general sense. Obviously it can be applied for any branch (subtree) of the main tree.

From this formulation, it can be seen that the denominator of the probability is a constant and can be extracted from the summation. In addition, if we check each inner-node till reaching a leaf, depth change is adding only one. Therefore, calculation of the APD can be reformulated as:

$$
A(T) = \frac{1}{\left|\sigma\right|}\sum\limits_{\forall n \in T}(\sigma(n))
$$

This simpler form of APD requires summing the number of samples in each inner-node within the tree (or branch). Since the number of total samples in a tree does not change during the pruning, it is easier to remove the denominator and use the absolute value for the APD evaluation.

<a name="2.2"></a>
## 2.2 - Benefit of using APD

While traditional tree pruning methods use number of nodes or leaves as a cost factor, here you can see the benefit of using APD to capture the effect of samples distribution in the tree and topology of the tree.
In the below figures both network have similar number of nodes and leaves.

<div> <img src="Figures/Topology1.png" alt="A DT" style="width: 200px;"/></div> 
<div> <img src="Figures/Topology2.png" alt="A DT" style="width: 200px;"/></div> 

Although both trees have 5 nodes and 6 leaves, their APD is different, First tree has a 2.7 APD, and the second one has an APD of 2.8.

<a name="3"></a>
## 3 - ORAP algorithm

Challenge of searching for the global optimal solution(s).

Overall algorithm

<a name="4"></a>
## 4 - ORAP performance

Changes in the accuracy according to the APD and number of nodes.

<a name="5"></a>
## 5 - ORAP vs CCP

Comparison of ORAP and CCP according to the APD and number of nodes.